# Vision Language Model Evaluation

This notebook evaluates multiple vision language models (e.g., LLaVA-NeXT, GPT-5) on three systematically constructed image datasets designed to probe color-concept representations: canonical object colors, counterfactual recolorings and abstract colored shapes.


In [3]:
%reload_ext autoreload
%autoreload 2

from transformers import (
    BitsAndBytesConfig,
    LlavaNextProcessor,
    LlavaNextForConditionalGeneration
)
import torch
import pandas as pd
import re
import gc
import numpy as np
from tqdm import tqdm
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt

ROOT = Path().resolve().parents[0]
sys.path.insert(0, str(ROOT))

from test_MLLMs import run_vlm_evaluation
from making_color_images.model_priors import TorchColorPriors, GPTColorPriors
from making_color_images.plot_variants import collect_variants_for, show_variants_grid, plot_vlm_performance, variant_label
from making_color_images.recolor_images import generate_variants, resize_all_images_and_masks


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

fontsize = 14

In [4]:
# Base data folder
WORK = Path(os.environ.get("WORK", Path.cwd()))
DATA = WORK / "color-concept-entanglement" / "data"

FRUIT = DATA / "fruit"
OUTLINES = DATA / "color_images"

RESIZED_IMGS = DATA / "resized_images"
RESIZED_MASKS = DATA / "resized_cv_masks"
RESIZED_IMGS.mkdir(parents=True, exist_ok=True)
RESIZED_MASKS.mkdir(parents=True, exist_ok=True)

In [6]:
# Set a specific seed for reproducibility
SEED = 42
rng = np.random.default_rng(SEED)

# Setting the seed for PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # If using GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True
)

torch.cuda.empty_cache()
gc.collect()

Using device: cuda


1013

In [16]:
# Load the datasets that where also used in the human evaluation
prior_df = pd.read_csv(DATA / "prolific_stimuli" / f"stimulus_table_image_priors_prolific.csv")
counterfact_df = pd.read_csv(DATA / "prolific_stimuli" / f"stimulus_table_counterfact_prolific.csv")
shape_df = pd.read_csv(DATA / "prolific_stimuli" / f"stimulus_table_shapes_prolific.csv")
display(prior_df.head(), prior_df.shape, counterfact_df.head(), counterfact_df.shape, shape_df.head(), shape_df.shape)

,image_path,object,stimulus_type,manipulation_color,target_color,variant_region,percent_colored,variant_label,mode
0,color_images/gpt-4o/image_priors/cheese_1_78f6...,cheese,correct_prior,yellow,yellow,BG,80,BG 80% (seq),seq
1,color_images/gpt-4o/image_priors/espresso_make...,espresso maker,correct_prior,red,red,FG,5,FG 5% (seq),seq
2,color_images/gpt-4o/image_priors/tile_roof_2_f...,tile roof,correct_prior,red,red,FG,100,FG 100% (seq),seq
3,color_images/gpt-4o/image_priors/cloud_3_29898...,cloud,correct_prior,grey,grey,FG,55,FG 55% (seq),seq
4,color_images/gpt-4o/image_priors/frilled_lizar...,frilled lizard,correct_prior,brown,brown,FG,100,FG 100% (seq),seq


(1260, 9)

,image_path,object,stimulus_type,manipulation_color,target_color,variant_region,percent_colored,variant_label,mode
0,color_images/gpt-4o/counterfact/rose_3_6471302...,rose,counterfact,blue,blue,FG,100,FG 100% (seq),seq
1,color_images/gpt-4o/counterfact/Sealyham_terri...,sealyham terrier,counterfact,purple,purple,FG,60,FG 60% (seq),seq
2,color_images/gpt-4o/counterfact/iguana_2_a2663...,iguana,counterfact,orange,orange,FG,10,FG 10% (seq),seq
3,color_images/gpt-4o/counterfact/hartebeest_3_5...,hartebeest,counterfact,red,red,FG,55,FG 55% (seq),seq
4,color_images/gpt-4o/counterfact/mouse_2_cf4ddb...,mouse,counterfact,red,red,FG,20,FG 20% (seq),seq


(412, 9)

,image_path,object,stimulus_type,manipulation_color,target_color,variant_region,percent_colored,variant_label,mode
0,shapes/shape_colored/hexagon_v3_yellow/FG_060_...,hexagon,shape,yellow,yellow,FG,60,FG 60% (seq),seq
1,shapes/shape_colored/pentagon_v0_purple/FG_055...,pentagon,shape,purple,purple,FG,55,FG 55% (seq),seq
2,shapes/shape_colored/square_v3_blue/FG_005_seq...,square,shape,blue,blue,FG,5,FG 5% (seq),seq
3,shapes/shape_colored/triangle_v0_brown/FG_050_...,triangle,shape,brown,brown,FG,50,FG 50% (seq),seq
4,shapes/shape_colored/pentagon_v2_orange/FG_090...,pentagon,shape,orange,orange,FG,90,FG 90% (seq),seq


(1331, 9)

In [12]:
def evaluate_vlm_on_stimuli(
    df_stimuli: pd.DataFrame,
    *,
    processor,
    model,
    device,
    image_root: Path,
    batch_size: int = 1,
    mode: str = "this",
    desc: str = "Evaluating stimuli",
):
    """
    Evaluate a VLM on a set of stimulus images.

    Expects df_stimuli to have columns:
      - image_path (relative to image_root)
      - object
      - target_color
    """

    preds = []

    for _, row in tqdm(
        df_stimuli.iterrows(),
        total=len(df_stimuli),
        desc=desc,
    ):

        df_eval_input = pd.DataFrame({
            "image_path": [image_root / row["image_path"]],
            "object": [row["object"]],
            "correct_answer": [row["target_color"]],
        })

        df_eval = run_vlm_evaluation(
            df=df_eval_input,
            processor=processor,
            model=model,
            device=device,
            batch_size=batch_size,
            mode=mode,
            return_probs=True,
        )

        preds.append(
            df_eval[[
                "object",
                "image_path",
                "correct_answer",
                "pred_color_this",
                "prob_correct_this",
            ]]
        )

        # GPU hygiene (important for long runs)
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        gc.collect()

    return pd.concat(preds, ignore_index=True)

# 1. LLaVA-NeXT

In [ ]:
model_name = "llava-v1.6-mistral-7b-hf"
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", quantization_config=bnb_config
).to(device)

LLAVA = DATA / "LLaVA-NeXT_results"
LLAVA.mkdir(parents=True, exist_ok=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## 1.1. Color Prior Dataset (LLaVA-NeXT)

In [14]:
suffix = "image_priors"
shape_pred_df = evaluate_vlm_on_stimuli(
    prior_df,
    processor=processor,
    model=model,
    device=device,
    image_root=DATA,
    desc="Evaluating canonical object colors (correct-prior stimuli)",
)

out_path = LLAVA / f"evaluation_shapes_{model_name}_{suffix}.csv"
shape_pred_df.to_csv(out_path, index=False)


Evaluating canonical object colors (correct-prior stimuli):   0%|          | 4/1260 [00:10<54:45,  2.62s/it]


KeyboardInterrupt: 

## 1.2. Counterfact Color Dataset (LLaVA-NeXT)

In [ ]:
suffix = "counterfact"
shape_pred_df = evaluate_vlm_on_stimuli(
    counterfact_df,
    processor=processor,
    model=model,
    device=device,
    image_root=DATA,
    desc="Evaluating counterfact object colors (counterfact stimuli)",
)

out_path = LLAVA / f"evaluation_shapes_{model_name}_{suffix}.csv"
shape_pred_df.to_csv(out_path, index=False)

## 1.3. Shape Dataset (LLaVA-NeXT)

In [ ]:
suffix = "shapes"
shape_pred_df = evaluate_vlm_on_stimuli(
    shape_df,
    processor=processor,
    model=model,
    device=device,
    image_root=DATA,
    desc="Evaluating shapes",
)

out_path = LLAVA / f"evaluation_shapes_{model_name}_{suffix}.csv"
shape_pred_df.to_csv(out_path, index=False)

# 2. GPT-5o

In [17]:
model_name = "gpt-5o"
GPT = DATA / "GPT_results"
GPT.mkdir(parents=True, exist_ok=True)